# Jupyter adaptation of run_svr.py

### imports

In [1]:
# General imports
import pandas as pd
import numpy as np
import os
import csv
import subprocess
import time
import shutil

# SciKit-Optimize:
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.utils import use_named_args

#
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
import pickle

# Misc. imports:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from scipy import stats
import statistics
import pickle

/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


### Global variables

In [2]:
dataset_path = '~/Dropbox/FreeSolv/dGlearn-FreeSolv-master/datasets/train_compiled/dGhydr_train.csv'
offset_col_name = 'dGoffset (kcal/mol)'

# set data processing configurations:
PCA_threshold = 0.95  # Keeps n dimensions for x variance explained
replicates = 30  # Number of replicates per subject model
n_calls = 40  # Number of Bayesian optimisation loops for hyperparameter optimisation, 40 is best for convergence, > 60 scales to very expensive
startpoint_BO = np.inf  # Point to consider top-performing model from (MAE/MAD); 1.0 = no improvement on test-set variance
ensemble_size = 10  # Amount of top-scoring models to retain per fold-dataset combination
# KFold parameters:
n_splits = 5  # Number of K-fold splits
random_state = 2  # Random number seed

split = 'dG(hydr)'
translated_subject = 'absolute'

### Read raw compiled data

In [3]:
# construct raw dataset
raw_data = pd.read_csv(dataset_path, index_col='ID')
raw_data

/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (272,382,383,384,385,386,387,388,389,390,481,482,483,589,590,591,592,593,594,595,596,597,697,698,699,778,779,794,795,874,875,890,891,920,921,935,948,961,974,987,1000,1013,1026,1089,1115,1123,1124,1131,1139,1140,1188,1531,1532,1533,1534,1555,1556,1605,2021,2049,2050,2052,2080) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,pfp0,pfp1,pfp2,pfp3,pfp4,pfp5,pfp6,pfp7,pfp8,pfp9,...,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,PBF,dGhydr (kcal/mol),uncertainty (kcal/mol)
ID,,,,,,,,,,,,,,,,,,,,,
mobley_5852491,0,0,0,0,0,0,0,0,0,0,...,4.671883,26,2,26.0,27.0,2.111111111111111,1.416667,1.576244e-01,0.19,0.60
mobley_9838013,0,0,0,0,0,0,0,0,0,0,...,4.893823,28,3,24.0,22.0,4.3125,1.375000,1.705255e-01,1.50,0.60
mobley_2410897,0,0,0,0,0,0,0,0,0,0,...,4.505785,4,0,6.0,4.0,2.25,1.000000,8.799730e-02,1.34,0.60
mobley_1893815,0,0,0,0,0,0,0,0,0,0,...,20.736383,28,3,24.0,22.0,4.3125,1.375000,5.848740e-01,1.34,0.10
mobley_2008055,0,0,0,0,0,0,0,0,0,0,...,3.755869,1,0,2.0,1.0,2.0,1.000000,3.361027e-17,0.63,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mobley_4149784,0,0,0,0,0,0,1,0,0,0,...,11.453386,414,25,88.0,104.0,4.66667,3.388889,2.726368e-04,0.35,0.13
mobley_6497672,0,0,0,0,0,0,0,0,0,0,...,6.367442,20,2,14.0,12.0,2.75,1.500000,1.585811e-04,0.88,0.60
mobley_9534740,0,0,0,0,0,0,0,0,0,0,...,7.502641,182,19,58.0,68.0,6.05556,2.777778,4.631200e-01,7.38,0.23


### Remove columns with string values

In [4]:
def check_dataframe_is_numeric(dataframe):
    """Iterate over all columns and check if numeric.

    Returns:
    New DataFrame with removed"""

    columns_dropped = 0
    columns_dropped_lst = []

    for col in dataframe.columns:
        for x in dataframe.loc[:, col]:
            try:
                float(x)
            except ValueError:
                columns_dropped_lst.append(col)
                columns_dropped += 1
                dataframe = dataframe.drop(columns=col)
                break

    print('Number of columns dropped:', (columns_dropped))
    return dataframe, columns_dropped_lst


numeric_data, columns_dropped = check_dataframe_is_numeric(raw_data)
numeric_data

Number of columns dropped: 712


,pfp0,pfp1,pfp2,pfp3,pfp4,pfp5,pfp6,pfp7,pfp8,pfp9,...,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2,PBF,dGhydr (kcal/mol),uncertainty (kcal/mol)
ID,,,,,,,,,,,,,,,,,,,,,
mobley_5852491,0,0,0,0,0,0,0,0,0,0,...,84.093900,4.671883,26,2,26.0,27.0,1.416667,1.576244e-01,0.19,0.60
mobley_9838013,0,0,0,0,0,0,0,0,0,0,...,88.088815,4.893823,28,3,24.0,22.0,1.375000,1.705255e-01,1.50,0.60
mobley_2410897,0,0,0,0,0,0,0,0,0,0,...,45.057849,4.505785,4,0,6.0,4.0,1.000000,8.799730e-02,1.34,0.60
mobley_1893815,0,0,0,0,0,0,0,0,0,0,...,165.891061,20.736383,28,3,24.0,22.0,1.375000,5.848740e-01,1.34,0.10
mobley_2008055,0,0,0,0,0,0,0,0,0,0,...,30.046950,3.755869,1,0,2.0,1.0,1.000000,3.361027e-17,0.63,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mobley_4149784,0,0,0,0,0,0,1,0,0,0,...,251.974485,11.453386,414,25,88.0,104.0,3.388889,2.726368e-04,0.35,0.13
mobley_6497672,0,0,0,0,0,0,0,0,0,0,...,70.041865,6.367442,20,2,14.0,12.0,1.500000,1.585811e-04,0.88,0.60
mobley_9534740,0,0,0,0,0,0,0,0,0,0,...,180.063388,7.502641,182,19,58.0,68.0,2.777778,4.631200e-01,7.38,0.23


### Convert all values to float

In [5]:
float_data = numeric_data.apply(pd.to_numeric).astype(float).sample(frac=1)
float_data = float_data.rename(columns={'dGhydr (kcal/mol)': 'dGoffset (kcal/mol)'})
float_data

,pfp0,pfp1,pfp2,pfp3,pfp4,pfp5,pfp6,pfp7,pfp8,pfp9,...,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2,PBF,dGoffset (kcal/mol),uncertainty (kcal/mol)
ID,,,,,,,,,,,,,,,,,,,,,
mobley_3976574,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,353.857568,16.084435,632.0,36.0,106.0,130.0,4.027778,0.000316,1.84,1.00
mobley_4780078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,122.073165,6.424903,84.0,10.0,42.0,46.0,2.027778,0.000201,1.03,0.60
mobley_7794077,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,180.014662,12.000977,186.0,16.0,60.0,67.0,2.527778,0.357283,1.89,0.16
mobley_2198613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,64.007978,8.000997,4.0,0.0,6.0,4.0,1.000000,0.115035,1.41,0.60
mobley_5520946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,110.019021,8.463002,42.0,5.0,30.0,31.0,1.666667,0.000106,1.05,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mobley_6620221,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,...,221.105193,7.132426,425.0,22.0,84.0,97.0,3.472222,0.296601,1.52,0.30
mobley_5857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,129.057849,7.591638,109.0,12.0,50.0,57.0,2.277778,0.000068,0.73,0.60
mobley_8746821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,107.073499,6.298441,60.0,8.0,36.0,39.0,1.861111,0.000175,1.45,0.60


### Normalise data and seperate labels before PCA

In [6]:
def normalise_and_split_datasets(collection):

    # process input dataset
    train_dataset = collection

    print('Normalising...')
    # Calculate statistics, compute Z-scores, clean:
    stats = train_dataset.describe()

    stats.pop('dGoffset (kcal/mol)')
    stats.pop('uncertainty (kcal/mol)')
    stats = stats.transpose()

    train_labels = train_dataset.pop('dGoffset (kcal/mol)')
    train_dataset.pop('uncertainty (kcal/mol)')

    def norm(x):
        return (x - stats['mean']) / stats['std']

    # Normalise and return separately:
    normed_train_data = norm(train_dataset).fillna(0).replace([np.inf, -np.inf], 0.0)

    return [normed_train_data, train_labels]


normalised_X, y_tmp = normalise_and_split_datasets(float_data)

Normalising...


### Perform PCA on features alone

In [7]:
def reduce_features(normalised_collection, pca_threshold):

    print('Computing PCA, reducing features up to ' + str(round(pca_threshold * 100, 5)) + '% VE..')
    training_data = normalised_collection

    # Initialise PCA object, keep components up to x% variance explained:
    PCA.__init__
    pca = PCA(n_components=pca_threshold)

    # Fit to and transform training set:
    train_post_pca = pd.DataFrame(pca.fit_transform(training_data))

    print('# of PCA features after reduction: ' + str(len(train_post_pca.columns)))

    train_post_pca.index = training_data.index
    # pickle pca object to file so that external test sets can be transformed accordingly
    # (see https://stackoverflow.com/questions/42494084/saving-large-data-set-pca-on-disk
    # -for-later-use-with-limited-disc-space)
    pickle.dump(pca, open('./opt_output/pca_trainingset.p', 'wb'))

    return train_post_pca  # return list with test_post_pca when needed


reduced_X = reduce_features(normalised_X, PCA_threshold)
reduced_X

Computing PCA, reducing features up to 95.0% VE..
# of PCA features after reduction: 107


,0,1,2,3,4,5,6,7,8,9,...,97,98,99,100,101,102,103,104,105,106
ID,,,,,,,,,,,,,,,,,,,,,
mobley_3976574,44.520072,23.164218,-6.762143,8.832869,1.801975,-0.695599,22.695296,5.331351,-2.098773,0.694682,...,0.119012,-0.113140,-1.969231,-0.221723,0.144667,1.209375,0.425151,0.330675,-0.318026,0.658045
mobley_4780078,1.587041,2.627165,-3.602003,-8.680417,-0.826186,0.453436,-3.648494,-1.253473,2.612930,2.327245,...,0.273075,0.224103,0.074727,-0.308314,-0.183456,-0.319635,-0.265054,0.495734,0.413026,0.373082
mobley_7794077,13.986763,9.540483,24.484462,-9.742516,-6.868446,2.359859,-3.927730,-5.579277,0.690055,-10.196704,...,1.208495,-4.156775,-3.494870,1.962051,2.538355,-2.205990,-0.280944,0.792237,0.296152,-1.811109
mobley_2198613,-23.163477,1.066900,1.357224,6.364280,2.282636,-4.011930,1.171974,5.090502,1.737021,0.788263,...,-0.330170,0.484245,-0.226300,0.055772,0.134273,-0.155175,0.095256,-0.318443,-0.226291,-0.311015
mobley_5520946,-7.287294,6.807771,-6.189251,-4.105125,1.827180,-3.844886,-3.589729,0.379780,0.631616,0.332880,...,-0.873820,-0.599095,0.724478,-0.224604,-2.220455,1.213225,-0.145380,0.049512,-0.252097,-0.668331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mobley_6620221,35.950867,-6.956677,1.687549,-3.684804,-7.271215,0.620817,0.637008,9.097634,1.190478,-10.412440,...,-0.904109,0.871035,-1.479719,2.671675,-2.973298,0.663362,4.218277,-1.460877,-0.889098,0.995973
mobley_5857,4.165705,8.000507,-8.783474,-11.791247,-1.722143,-3.247366,-2.584409,3.183495,3.073314,-4.215161,...,1.238470,-0.177060,4.626908,-3.762473,-1.339345,0.026715,2.168524,-2.595233,2.646946,-0.192624
mobley_8746821,-2.722488,2.021294,-4.214061,-8.768638,-2.496633,-1.506943,-3.302950,1.455893,2.992012,-0.764604,...,1.010303,-0.386447,0.257594,-1.348849,0.086837,0.040262,-0.384397,-0.031386,0.226480,-0.266740


### Recombine reduced features with labels with the correct indexing

In [8]:
dataset = pd.concat([reduced_X, y_tmp], axis=1)
dataset

,0,1,2,3,4,5,6,7,8,9,...,98,99,100,101,102,103,104,105,106,dGoffset (kcal/mol)
ID,,,,,,,,,,,,,,,,,,,,,
mobley_3976574,44.520072,23.164218,-6.762143,8.832869,1.801975,-0.695599,22.695296,5.331351,-2.098773,0.694682,...,-0.113140,-1.969231,-0.221723,0.144667,1.209375,0.425151,0.330675,-0.318026,0.658045,1.84
mobley_4780078,1.587041,2.627165,-3.602003,-8.680417,-0.826186,0.453436,-3.648494,-1.253473,2.612930,2.327245,...,0.224103,0.074727,-0.308314,-0.183456,-0.319635,-0.265054,0.495734,0.413026,0.373082,1.03
mobley_7794077,13.986763,9.540483,24.484462,-9.742516,-6.868446,2.359859,-3.927730,-5.579277,0.690055,-10.196704,...,-4.156775,-3.494870,1.962051,2.538355,-2.205990,-0.280944,0.792237,0.296152,-1.811109,1.89
mobley_2198613,-23.163477,1.066900,1.357224,6.364280,2.282636,-4.011930,1.171974,5.090502,1.737021,0.788263,...,0.484245,-0.226300,0.055772,0.134273,-0.155175,0.095256,-0.318443,-0.226291,-0.311015,1.41
mobley_5520946,-7.287294,6.807771,-6.189251,-4.105125,1.827180,-3.844886,-3.589729,0.379780,0.631616,0.332880,...,-0.599095,0.724478,-0.224604,-2.220455,1.213225,-0.145380,0.049512,-0.252097,-0.668331,1.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mobley_6620221,35.950867,-6.956677,1.687549,-3.684804,-7.271215,0.620817,0.637008,9.097634,1.190478,-10.412440,...,0.871035,-1.479719,2.671675,-2.973298,0.663362,4.218277,-1.460877,-0.889098,0.995973,1.52
mobley_5857,4.165705,8.000507,-8.783474,-11.791247,-1.722143,-3.247366,-2.584409,3.183495,3.073314,-4.215161,...,-0.177060,4.626908,-3.762473,-1.339345,0.026715,2.168524,-2.595233,2.646946,-0.192624,0.73
mobley_8746821,-2.722488,2.021294,-4.214061,-8.768638,-2.496633,-1.506943,-3.302950,1.455893,2.992012,-0.764604,...,-0.386447,0.257594,-1.348849,0.086837,0.040262,-0.384397,-0.031386,0.226480,-0.266740,1.45


### Perform 5-fold cross-validation

In [9]:
def split_dataset(dataset, n_splits, random_state):
    """KFold implementation for pandas DataFrame.
    (https://stackoverflow.com/questions/45115964/separate-pandas-dataframe-using-sklearns-kfold)"""
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    kfolds = []
    global offset_col_name
    
    for train, test in kf.split(dataset):

        training = dataset.iloc[train]
        train_labels = training[offset_col_name]
        train_set = training.drop(offset_col_name, axis=1)

        testing = dataset.iloc[test]
        test_labels = testing[offset_col_name]
        test_set = testing.drop(offset_col_name, axis=1)

        kfolds.append(
        [[train_set, test_set],
        [train_labels, test_labels]]
        )
    
    return kfolds


kfolds = split_dataset(dataset, n_splits, random_state)
kfolds

[[[                      0          1          2          3         4    \
   ID                                                                     
   mobley_3976574  44.520072  23.164218  -6.762143   8.832869  1.801975   
   mobley_7794077  13.986763   9.540483  24.484462  -9.742516 -6.868446   
   mobley_5520946  -7.287294   6.807771  -6.189251  -4.105125  1.827180   
   mobley_5852491 -10.278650  -4.657391  -8.654960   2.217482  1.312801   
   mobley_8772587 -13.117957   2.072349  -6.409431  13.614349  2.789836   
   ...                   ...        ...        ...        ...       ...   
   mobley_4043987  -6.335918  -9.764654  -9.253077   3.655430  1.148478   
   mobley_9557440   1.570568  11.405998  11.994993  -8.841855 -5.335803   
   mobley_5857      4.165705   8.000507  -8.783474 -11.791247 -1.722143   
   mobley_8746821  -2.722488   2.021294  -4.214061  -8.768638 -2.496633   
   mobley_2123854   0.041200   8.148822  -0.160674  -9.521529  1.419318   
   
                     

In [10]:
for fold in kfolds:
    dataframe = fold
    train_postPCA_df, test_postPCA_df, train_labels_df, test_labels_df = dataframe[0][0], dataframe[0][1], dataframe[1][0], dataframe[1][1]
    train_postPCA = train_postPCA_df.astype(np.float32).values
    test_postPCA = test_postPCA_df.astype(np.float32).values
    train_labels = train_labels_df.astype(np.float32).values
    test_labels = test_labels_df.astype(np.float32).values

### Define SVR

In [21]:
def svr(dataframe, dataset_name, iteration):

    model_bucket = []
    
#     [[train_set, test_set], [train_labels, test_labels]]

    # Retrieve datasets, convert to float32 for RF:
    train_postPCA_df, test_postPCA_df, train_labels_df, test_labels_df = dataframe[0][0], dataframe[0][1], dataframe[1][0], dataframe[1][1]
    train_postPCA = train_postPCA_df.astype(np.float32).values
    test_postPCA = test_postPCA_df.astype(np.float32).values
    train_labels = train_labels_df.astype(np.float32).values
    test_labels = test_labels_df.astype(np.float32).values

    # Set hyperparameter ranges, append to list:
    dim_param_C = Categorical(categories=list(np.logspace(-3, 2, 6, dtype="float32")), name="param_C")
    dim_param_gamma = Categorical(categories=list(np.logspace(-3, 2, 6, dtype="float32")), name="param_gamma")
    dim_param_epsilon = Categorical(categories=list(np.logspace(-3, 2, 6, dtype="float32")), name="param_epsilon")

    dimensions = [dim_param_C, dim_param_gamma, dim_param_epsilon]	
    print("###################################")
    print("Fold is:", iteration)
    @use_named_args(dimensions=dimensions)
    def fitness(param_C, param_gamma, param_epsilon):
    # Create the svm with these hyper-parameters:


        svm_estimator = SVR(gamma=param_gamma, C=param_C, epsilon=param_epsilon)
        svm_estimator.fit(train_postPCA, train_labels)  

        prediction_list = svm_estimator.predict(test_postPCA)

        # calculate some statistics on test set:
        MAE = mean_absolute_error(test_labels, prediction_list)
        MAD_testset = test_labels_df.mad()

        MAEMAD = MAE/MAD_testset
        print("MAE/MAD:",MAEMAD, "Fold:", iteration)

        perts_list = test_labels_df.index.tolist()
        exp_list = test_labels_df.values.tolist()

        slope, intercept, r_value, p_value, std_err = stats.linregress(prediction_list, exp_list)
        tau, p_value = stats.kendalltau(prediction_list, exp_list)


        # For plotting test set correlations:
        tuples_result = list(zip(perts_list, exp_list, prediction_list))
        nested_list_result = [ list(elem) for elem in tuples_result ]

    # Append data with best performing model.
    # Data contains the MAE/MAD score, protein target, iteration,
    # tau, r value, the keras DNN model, the internal validation plot 
    # and the data for external validation:

        global startpoint_MAEMAD

        if MAEMAD < startpoint_MAEMAD:
            startpoint_MAEMAD = MAEMAD
            model_bucket.append([MAEMAD, dataset_name, iteration, tau, r_value, nested_list_result])

            # # write all model files:
            if not os.path.exists("./opt_tmp"):
                os.makedirs("./opt_tmp")

            with open("./opt_tmp/"+str(iteration)+"_ALFRESCO_TopPerform_SVM.svm", "wb") as file:
                pickle.dump(svm_estimator, file)


        return MAEMAD

    # Bayesian Optimisation to search through hyperparameter space. 
    # Prior parameters were found by manual search and preliminary optimisation loops. 
    # For running just dataset 13x500 calls, optimal hyperparameters from 150 calls were used as prior.
    default_parameters = [1.0, 1.0, 1.0]
    print("###########################################")
    print("Created model, optimising hyperparameters..")

    search_result= gp_minimize(func=fitness,
                                dimensions=dimensions,
                                acq_func='EI', #Expected Improvement.
                                n_calls=n_calls,
                                x0=default_parameters)


    print("###########################################")
    print("Concluded optimal hyperparameters:")
    print(search_result.x)

    print("###########################################")

    # return skopt object and highest scoring model for this replicate:
    return search_result, model_bucket[-1]

### Start writing log file

In [18]:
# initiate empty DF to fill with cumulative minima 
cumulative_MAEs = pd.DataFrame()
cumulative_MAEtauR_CV = pd.DataFrame()

# clean slate opt_output:
if os.path.exists("./opt_output"):
    subprocess.call('rm ./opt_output/*', shell=True)
if not os.path.exists("./opt_output"):
    os.mkdir("./opt_output")
    
# initiate log file:
with open("opt_output/logfile.txt", "w") as file:
    writer = csv.writer(file, delimiter='\t')
    writer.writerow(["###########Starting tb-CV BO.###########"])
    writer.writerow(["PCA threshold: "+str(PCA_threshold)])
    writer.writerow(["n replicates: "+str(replicates)])
    writer.writerow(["n models in ensemble: "+str(ensemble_size)])
    writer.writerow(["n calls (BO): "+str(n_calls)])
    writer.writerow(["Started program at: "+time.ctime()])

bucket_df = pd.DataFrame()
mae_results_per_fold = [["Subject", "MAE", "Replicate"]]
MAEtauR_results_per_fold = [["Subject", "Correlation Coefficient", "Dataset", "Correlation metric"]]

print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
print(time.ctime())

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Mon Dec  2 10:44:55 2019


In [22]:
fold_index = 1
for fold in kfolds:
    MAEs_per_fold = []
    models_per_replicate = []
    
    # run tb-CV:
    # reset MAEMAD startpoint per replicate:
    startpoint_MAEMAD = startpoint_BO
    OptimizeResult, top_performer = svr(fold, 'dGhydr', fold_index)

    models_per_replicate.append(top_performer)

    # construct, cummin and concatenate results of this replicate to the other replicates in the loop:
    split_columns = { 
        "Dataset" : str(split), 
        "MAE/MAD" : OptimizeResult.func_vals, 
        "Subject": translated_subject}
    result_df = pd.DataFrame(split_columns).cummin()
    bucket_df = pd.concat([bucket_df, result_df])
    # tag data with the dataset type (i.e. descriptor set), add to complete results:
    bucket_df["Dataset"] = str(split)
    cumulative_MAEs = pd.concat([cumulative_MAEs, bucket_df])


    # retrieve statistics for this replicate:					
    tau = top_performer[3]
    r_value = top_performer[4]
    MAE = top_performer[0]

    MAEtauR_results_per_fold.append([translated_subject, r_value, split, "Pearson's-r"])
    MAEtauR_results_per_fold.append([translated_subject, tau, split, "Kendall's-tau"])
    MAEtauR_results_per_fold.append([translated_subject, MAE, split, "MAE/MAD"])

    # write update to log file:
    with open("opt_output/logfile.txt", "a") as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerow(["Finished "+translated_subject+", dataset "+split+", replicate "+str(fold_index)+" at "+str(time.ctime())])
    fold_index += 1

###################################
Fold is: 1
###########################################
Created model, optimising hyperparameters..
MAE/MAD: 1.2183454530507065 Fold: 1
MAE/MAD: 0.9939922564555703 Fold: 1
MAE/MAD: 7.069537091295838 Fold: 1
MAE/MAD: 7.069537091295838 Fold: 1
MAE/MAD: 7.069537091295838 Fold: 1
MAE/MAD: 0.9699915324623829 Fold: 1
MAE/MAD: 0.994428104297651 Fold: 1
MAE/MAD: 7.069537091295838 Fold: 1


/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)
/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)
/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  s

MAE/MAD: 1.0197286979686888 Fold: 1
MAE/MAD: 0.9946175495792587 Fold: 1
MAE/MAD: 7.069537091295838 Fold: 1


/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


MAE/MAD: 1.0401338510577223 Fold: 1
MAE/MAD: 0.9918177076755827 Fold: 1
MAE/MAD: 0.8464207886402352 Fold: 1
MAE/MAD: 0.7844339475732858 Fold: 1
MAE/MAD: 0.8573860369570184 Fold: 1
MAE/MAD: 0.9946278707043175 Fold: 1
MAE/MAD: 0.7627226098725637 Fold: 1


...

MAE/MAD: 0.6149963579246042 Fold: 4
MAE/MAD: 0.8140806217606578 Fold: 4
MAE/MAD: 0.9881559174831792 Fold: 4
MAE/MAD: 1.014631796870271 Fold: 4
MAE/MAD: 0.9891177023332146 Fold: 4


/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


MAE/MAD: 0.6149963579246042 Fold: 4
MAE/MAD: 1.015693079972531 Fold: 4
MAE/MAD: 0.631135155105905 Fold: 4
MAE/MAD: 0.7788116901533385 Fold: 4
MAE/MAD: 0.9830462439112988 Fold: 4
###########################################
Concluded optimal hyperparameters:
[100.0, 0.001, 0.1]
###########################################
###################################
Fold is: 5
###########################################
Created model, optimising hyperparameters..
MAE/MAD: 1.1117957460352912 Fold: 5
MAE/MAD: 1.1117957460352912 Fold: 5
MAE/MAD: 0.9733703811854447 Fold: 5
MAE/MAD: 0.6494529407902037 Fold: 5
MAE/MAD: 0.9660522843829006 Fold: 5
MAE/MAD: 3.4746065978537035 Fold: 5
MAE/MAD: 1.0896063801486435 Fold: 5
MAE/MAD: 0.9659538910924387 Fold: 5
MAE/MAD: 1.1117957460352912 Fold: 5
MAE/MAD: 3.4746065978537035 Fold: 5
MAE/MAD: 3.4746065978537035 Fold: 5


/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)
/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)
/Users/wilsonwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  s

MAE/MAD: 0.973844295051722 Fold: 5
MAE/MAD: 0.9661752921117486 Fold: 5
MAE/MAD: 0.9411845623450743 Fold: 5
MAE/MAD: 0.9729584394065388 Fold: 5
MAE/MAD: 0.6899045197137978 Fold: 5
MAE/MAD: 0.641008819321725 Fold: 5
MAE/MAD: 0.9633006725637597 Fold: 5
MAE/MAD: 0.9571723119701195 Fold: 5
MAE/MAD: 0.9777438283545251 Fold: 5
MAE/MAD: 0.9630016299149113 Fold: 5
MAE/MAD: 0.9649980187792109 Fold: 5
MAE/MAD: 0.8567406108304171 Fold: 5
MAE/MAD: 0.9773793897128358 Fold: 5
MAE/MAD: 0.9650631205258405 Fold: 5
MAE/MAD: 0.9636172935174587 Fold: 5
MAE/MAD: 0.7766416467564087 Fold: 5
MAE/MAD: 0.9596615848107736 Fold: 5
MAE/MAD: 0.9736379935147721 Fold: 5
MAE/MAD: 0.775139887907762 Fold: 5
MAE/MAD: 0.6587536957460397 Fold: 5
MAE/MAD: 0.9735801993515385 Fold: 5
MAE/MAD: 0.858351592298504 Fold: 5
MAE/MAD: 0.9401616908598606 Fold: 5
MAE/MAD: 0.7940538820511153 Fold: 5
MAE/MAD: 0.8978432064262398 Fold: 5
MAE/MAD: 0.6386055953937184 Fold: 5
MAE/MAD: 0.7774818667547572 Fold: 5
MAE/MAD: 0.6909624388609539 Fold

In [23]:
# make ensemble of best models; pick n replicates' top performing models:

models_per_replicate = sorted(models_per_replicate, key=lambda x: x[0])

ensemble_collection = models_per_replicate[:ensemble_size]

i=1
for best_model_collection in ensemble_collection:

    opt_replicate = str(best_model_collection[2])
    result_internal = MAE

    nested_list_result_external = best_model_collection[5]


# For this best model, retrieve model files, plot internal validation and write external validation to file:
    if not os.path.exists("./opt_output"):
        os.mkdir("./opt_output")

    # with the known optimal replicate #, isolate model files from opt_tmp and move to opt_output:
    # rename so that name contains name of the feature set instead of the replicate:
    os.rename(
        "opt_tmp/"+opt_replicate+"_ALFRESCO_TopPerform_SVM.svm",
        "opt_output/model"+str(i)+"_"+split+"_"+translated_subject+"_ALFRESCO_TopPerform_SVM.svm"
        )

    i+=1
# to keep things clean, remove ./opt_tmp:
shutil.rmtree("./opt_tmp/")

# write internal validation MAEMAD value:
internal_val = pd.DataFrame([result_internal], columns=["val_loss"])

internal_val.to_csv("opt_output/"+str(split)+"_"+str(translated_subject)+"_TopPerformer_internalVal_df.csv")

# write external validation DF:
with open("opt_output/"+str(split)+"_"+str(translated_subject)+"_TopPerformer_externalVal_df.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(["Perturbation", "Experimental ddGoffset (kcal/mol)", "Predicted ddGoffset (kcal/mol)", "Subject"])
    for row in nested_list_result_external:
        writer.writerow(row + [translated_subject])

In [24]:
MAEs_CV = pd.DataFrame(mae_results_per_fold[1:], columns=mae_results_per_fold[0])
MAEtauR_CV = pd.DataFrame(MAEtauR_results_per_fold[1:], columns=MAEtauR_results_per_fold[0])
cumulative_MAEtauR_CV = pd.concat([cumulative_MAEtauR_CV, MAEtauR_CV])

In [25]:
cumulative_MAEtauR_CV.to_csv("opt_output/tb-CV_MAEtauR_outputs.csv", index=False)
cumulative_MAEs.to_csv("opt_output/tbCV_BO_MAE.csv")
print("Success, wrote all files to opt_output/.")

Success, wrote all files to opt_output/.
